# Введение в программирование на языке Python
## Семинар 7. Повторение

#### Задача 1 (3, Семинар 6)

Предположим, что мы не только хотим прочитать таблицу из файла (или с HTML-страницы), но еще и записать ее в нашу базу данных. Соответственно, эту таблицу необходимо создать. Для этого нужен примерно такой код:

```
CREATE TABLE distributors (
    did integer,
    name varchar(40)
);
```

PostgreSQL [располагает](https://www.postgresql.org/docs/current/datatype.html) следующими типами данных (это не все, что есть, но все, что нам сейчас пригодится):

<table>
    <tr>
        <th>Python</th>
        <th>PostgreSQL</th>
    </tr>
    <tr>
        <td>int</td>
        <td>smallint, integer, bigint</td>
    </tr>
    <tr>
        <td>float</td>
        <td>real</td>
    </tr>
    <tr>
        <td>bool</td>
        <td>boolean</td>
    </tr>
    <tr>
        <td>str</td>
        <td>varchar, text</td>
    </tr>
</table>

Реализуйте соответствующую функцию. Она должна иметь такие аргументы:

- `filename` (строка);
- `sep` (строка, по дефолту для CSV);
- `tablename` (строка, по дефолту соответствует названию файла без расширения);
- `varnames` (либо должны быть предоставлены в виде списка, либо добываются из данных - аргумент `use_first`);
- `use_first` (использовать ли первую строку файла в качестве названий колонок. Логический, по дефолту да);
- `dtypes` (либо должны быть предоставлены в виде списка, либо добываются из данных - аргумент `infer_dtypes`);
- `infer_dtypes` (попытаться ли угадать тип данных из файла. Логический, по дефолту нет).

In [ ]:
# если [True, False, true, false]:
## bool +
# иначе 
## если это текст:
### или varchar, text +?
## иначе
### если есть точка "."
#### float (real) +
### иначе
#### smallint, integer, bigint +?


In [ ]:
# varchar or text
# see documantation ()

In [ ]:
# smallint or integer or bigint
# see documantation ()

In [ ]:
# -{0,1}[digits].{0,1}[digits]{0,N}

In [24]:
def check_bool(column: list[str]) -> bool:
    bool_values = ['true', 'false']
    column_new = [value.lower() for value in column]
    indicators = [value in bool_values for value in column_new]
    
    return all(indicators)

def check_numeric_value(string: str) -> bool:
    verdict = False
    
    minus_count = string.count('-')
    
    if string.count('.') > 1:
        pass
    else:
        if minus_count > 1:
            pass
        elif minus_count == 1 and string.startswith('-'):
            string_new = string.replace('-', '').replace('.', '')
        
            if string_new.isdigit():
                verdict = True
                
        elif minus_count < 1:
            string_new = string.replace('-', '').replace('.', '')
            
            if string_new.isdigit():
                verdict = True    
        
    return verdict

def check_numeric(column: list[str]) -> bool:
    indicator = [check_numeric_value(value) for value in column]
    
    return all(indicator)

def check_real(column: list[str]) -> bool:
    indicator = ['.' in value for value in column]
    
    return any(indicator)

def infer_dtype(column: list[str]) -> str:
    dtype = None
    
    # check bool
    bcheck = check_bool(column)
    
    if bcheck:
        dtype = 'bool'
    else:
        ncheck = check_numeric(column)  
        if ncheck:
            rcheck = check_real(column)
            if rcheck:
                dtype = 'real'   
            else:
                # check int
        else:
            # text or varchar
        
    return None

In [27]:
check_numeric_value('1000-')

False

In [23]:
all([True, True, True, False])

False

In [22]:
check_bool(sdict['Height'])

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


In [15]:
def infer_sql_from_file(
    filename: str,
    sep=',',
    tablename=None,
    varnames=None,
    dtypes=None
):
    # прочитаем файл
    file = open(filename, 'r')  # читаем файл
    text = file.read()  # достаем файл
    file.close()

    lines = text.strip().split('\n')  # делим текст на строки
    
    # создадим заголовки в зависимости от переменной 
    if varnames is None:
        headers = lines[0].replace('п»ї', '')  # извлекаем заголовки varnames
        headers = headers.split(sep)
    else:
        headers = varnames.copy()

    lists = []

    for line in lines[1:]:
        lists.append(line.split(sep))
    
    columns = list(zip(*lists))
    headers_columns = zip(headers, columns)
    super_dict = dict(headers_columns)
    
    # попробуем угадать типы
    
    
    # создадим название таблицы
    if tablename is None:
        dot_index = filename.rfind('.')
        tablename = filename[:dot_index]

    # сгенерируем запрос
    headers_dtypes = ',\n'.join([f'\t"{header}" {dtype}' for header, dtype in zip(headers, dtypes)])   
    request = f'CREATE TABLE {tablename} (\n{headers_dtypes}\n);'
    
    # сохораним в файл
    sql_filename = tablename + '.sql'
    with open(sql_filename, 'w') as outfile:
        outfile.write(request)
    
    return sql_filename, super_dict

In [21]:
dtypes = ['text', 'text', 'text', 'text', 'text', 'real', 'text', 'text', 'text', 'real']

_, sdict = infer_sql_from_file('Super_pp4.csv', sep=';', dtypes=dtypes)

In [ ]:
# 2 варианта:
#     file = open(filename, 'r')  # читаем файл
#     text = file.read()  # достаем файл
#     file.close()


# # outfile = open(sql_filename, 'w')
# with open(sql_filename, 'w') as outfile:
#     text = file.read()  # достаем файл

In [11]:
name, headers = infer_sql_from_file('Super_pp4.csv', sep=';')
dtypes = ['text', 'text', 'text', 'text', 'text', 'real', 'text', 'text', 'text', 'real']

headers_dtypes = ',\n'.join([f'\t"{header}" {dtype}' for header, dtype in zip(headers, dtypes)])

f'CREATE TABLE {name} (did integer, name varchar(40));'

TypeError: 'NoneType' object is not iterable

In [16]:
print(f'CREATE TABLE {name} (\n{headers_dtypes}\n);')

NameError: name 'name' is not defined

In [13]:
headers_dtypes

NameError: name 'headers_dtypes' is not defined

In [14]:
headers

NameError: name 'headers' is not defined

#### Задача 2. Вспомним рекурсию

Надо написать функцию, которая будет считать факториал с помощью рекурсии.

In [5]:
def factorial(x):
    if x <= 1:
        return 1
    else:
        return x * factorial(x-1)

In [ ]:
# x = 4

#0 factorial(4)
#1 factorial(4) factorial(3)
#2 factorial(4) factorial(3) factorial(2)
#3.1 factorial(4) factorial(3) factorial(2) factorial(1)
#3.2 factorial(4) factorial(3) factorial(2) 1
#4 factorial(4) factorial(3) 2 * 1
#5 factorial(4) 3 * 2 * 1
#6 4 * 3 * 2 * 1

Очередь и стек

In [ ]:
## очередь (first in - first out)
#T    begininning                  ->                     end
#1      object1
#2      object2, object1
#3      object3, object2, object1
#i
# i + 1 object3, object2                                 object1
# i + 2 object3                                     object2, object1
# i + 3    

In [ ]:
## стек (first in - last out)
#T    begininning                  ->                     end
#1      object1
#2      object1, object2
#3      object1, object2, object3
#i
# i + 1 object1, object2                                object3
# i + 2 object1                                     object2, object3
# i + 3                                         object1, object2, object3

**Проверим правильность написанной функции**

In [2]:
import math

In [6]:
numbers = range(15)

our_factorials = [factorial(i) for i in numbers]
their_factorials = [math.factorial(i) for i in numbers]

In [7]:
print(our_factorials)
print(their_factorials)

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800, 39916800, 479001600, 6227020800, 87178291200]
[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800, 39916800, 479001600, 6227020800, 87178291200]


**Задача 3** (взята [отсюда](https://pythontutor.ru/lessons/functions/problems/reverse_rec/))

Дана последовательность целых чисел, заканчивающаяся числом 0 (числа вводит пользователь). Выведите эту последовательность в обратном порядке (с помощью `print()`).
При решении этой задачи нельзя пользоваться массивами и прочими динамическими структурами данных. Рекурсия вам поможет.

In [9]:
def reverse_input():
    value = input()
    
    if value == '0':
        print(value)
    else:
        reverse_input()
        print(value)

In [11]:
reverse_input()

0
2
9
5


In [ ]:
#0 factorial(inp=5)
#1 factorial(inp=5) factorial(inp=9)
#2 factorial(inp=5) factorial(inp=9) factorial(inp=2)
#3.1 factorial(inp=5) factorial(inp=9) factorial(inp=2) factorial(inp=0)
#3.2 factorial(inp=5) factorial(inp=9) factorial(inp=2) print(0)
#4 factorial(inp=5) factorial(inp=9) print(2) print(0)
#5 factorial(inp=5) print(9) print(2) print(0)
#6 print(5) print(9) print(2) print(0)